<a href="https://colab.research.google.com/github/aswat98/DesignOptimization2021Fall/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import logging
import math
import random
import numpy as np
import time
import torch as t
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt

logger = logging.getLogger(__name__)

In [1]:
FRAME_TIME = 0.1  # second, time interval
GRAVITY_ACCEL = 10.0  # m/s2, graviational force, g
BOOST_ACCEL = 15.0  # m/s2, thrust constant

In [4]:
class Dynamics(nn.Module):
    def __init__(self):
        super(Dynamics, self).__init__()
    @staticmethod
    def forward(state, action):
        """
        action:
        action[0]: y directional boost       
        states:
        state[0] = y
        state[1] = vy  
        """   
        delta_state_gravity = t.tensor([0., GRAVITY_ACCEL * FRAME_TIME])
        thrust = action[0]
        delta_state_y = BOOST_ACCEL * FRAME_TIME * t.tensor([0., -1.0]) * thrust
        state = state + delta_state_y + delta_state_gravity
        step_mat = t.tensor([[1., FRAME_TIME], [0., 1.]])      
        state = t.matmul(step_mat, state)
        return state

In [5]:
class Dynamics(nn.Module):

    def __init__(self):
        super(Dynamics, self).__init__()

    @staticmethod
    def forward(state, action):

        """
        action: thrust or no thrust
        state[0] = y
        state[1] = y_dot
        """

        state[1] = state[1] + GRAVITY_ACCEL * FRAME_TIME - BOOST_ACCEL * FRAME_TIME * action
        
        
        step_mat = t.tensor([[1., FRAME_TIME],
                            [0., 1.]])
        state = t.matmul(step_mat, state)

        return state

In [6]:
class Controller(nn.Module):
    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden, dim_hidden),
            nn.Sigmoid()
        )
    def forward(self, state):
        action = self.network(state)
        return action